In [3]:
import meep as mp
import numpy as np
from meep.materials import Au
from matplotlib import pyplot as plt

def grating_simulation(period, nslits, t, w):
    resolution = 50         # pixels/μm                                                                                                                   
    
    dpml = 1.0              # PML thickness                                                                                                               
    dsub = 2.0              # substrate thickness                                                                                                         
    dpad = 2.0              # padding between grating and PML                                                                                             
    gp = period                # grating periodicity                                                                                                         
    gh = t                # grating height                                                                                                              
    gdc = 1.0               # grating duty cycle
    w = w
    
    num_cells = nslits
    gdc_list = [gdc for _ in range(num_cells)]
    
    wvl_min = 0.4
    wvl_max = 0.85
    fmin = 1 / wvl_max
    fmax = 1 / wvl_min
    fcen = 0.5 * (fmax + fmin)
    df = fmax - fmin
    nfreq = 200
    
    k_point = mp.Vector3()
    
    glass = mp.Medium(index=2)
    
    pml_layers = [mp.PML(thickness=dpml)]
    
    symmetries=[mp.Mirror(mp.Y)]
    
    sx = dpml+dsub+gh+dpad+dpml
    sy = dpml+num_cells*gp+dpml
    cell_size = mp.Vector3(sx,sy)
    
    src_pt = mp.Vector3(-0.5*sx+dpml+0.5*dpad)
    sources = [mp.Source(mp.GaussianSource(fcen, fwidth=df),
                         component=mp.Ey,
                         center=src_pt,
                         size=mp.Vector3(y=sy-2*dpml))]
    
    geometry = [mp.Block(material=glass,
                         size=mp.Vector3(dpml+dsub,mp.inf,mp.inf),
                         center=mp.Vector3(-0.5*sx+0.5*(dpml+dsub)))]
    
    sim = mp.Simulation(resolution=resolution,
                        cell_size=cell_size,
                        boundary_layers=pml_layers,
                        geometry=geometry,
                        k_point=k_point,
                        sources=sources,
                        symmetries=symmetries)
    
    incident_pt = mp.Vector3(0.5*sx-dpml-0.5*dpad)
    incident_region = mp.FluxRegion(center=incident_pt, size=mp.Vector3(y=sy-2*dpml))
    incident_flux_monitor = sim.add_flux(fcen,df,nfreq,incident_region)
    
    sim.run(until_after_sources=100)
    
    incident_flux_data = sim.get_flux_data(incident_flux_monitor)
    incident_flux = mp.get_fluxes(incident_flux_monitor)
    
    
    sim.reset_meep()
    
    for j in range(num_cells):
      geometry.append(mp.Block(material=Au,
                               size=mp.Vector3(gh,gdc_list[j]*gp-w,mp.inf),
                               center=mp.Vector3(-0.5*sx+dpml+dsub+0.5*gh,-0.5*sy+dpml+(j+0.5)*gp)))
    
    sim = mp.Simulation(resolution=resolution,
                        cell_size=cell_size,
                        boundary_layers=pml_layers,
                        geometry=geometry,
                        k_point=k_point,
                        sources=sources,
                        symmetries=symmetries)
    
    transmission_pt = mp.Vector3(0.5*sx-dpml-0.5*dpad)
    transmission_region = mp.FluxRegion(center=transmission_pt, size=mp.Vector3(y=sy-2*dpml))
    transmission_flux_monitor = sim.add_flux(fcen,df,nfreq,transmission_region)
    
    sim.run(until_after_sources=100)
    
    transmitted_flux = mp.get_fluxes(transmission_flux_monitor)
    T = abs(np.divide(transmitted_flux, incident_flux))
    
    freqs = mp.get_flux_freqs(transmission_flux_monitor)
    wvls = np.divide(1000, freqs)
    
    per = gp*1000
    plt.plot(wvls, T/T.max(), label='p = %d nm' % per)
    
    # Adicione um título ao gráfico
    plt.title('')
    
    # Adicione rótulos aos eixos
    plt.ylabel('Transmission (a.u.)')
    plt.xlabel('Wavelength (nm)')
    
    # Adicione uma legenda
    plt.legend()
    
    plt.savefig('p=%d'%per)

    plt.close()
    
    with open("p=%dnm.txt"%per, "w") as file:
        # Passo 3: Iterar sobre os elementos dos arrays e escrever no formato desejado
        for x, y in zip(wvls, T/T.max()):
            file.write(f"{x} {y}\n")  # Escreve no formato "x y" e adiciona uma nova linha
    return

In [4]:
grating_simulation(0.3, 66, 0.17, 0.06)
grating_simulation(0.4, 50, 0.17, 0.06)
grating_simulation(0.5, 40, 0.17, 0.06)
grating_simulation(0.7, 29, 0.17, 0.06)
grating_simulation(0.9, 23, 0.17, 0.06)

-----------
Initializing structure...
Halving computational cell along direction y
time for choose_chunkdivision = 0.000404119 s
Working in 2D dimensions.
Computational cell is 6.18 x 21.8 x 0 with resolution 50
     block, center = (-1.585,0,0)
          size (3,1e+20,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (4,4,4)
time for set_epsilon = 0.181029 s
-----------
Meep progress: 57.52/107.55555534362793 = 53.5% done in 4.0s, 3.5s to go
on time step 5752 (time=57.52), 0.000695486 s/step


run 0 finished at t = 107.56 (10756 timesteps)
-----------
Initializing structure...
Halving computational cell along direction y
time for choose_chunkdivision = 0.000518084 s
Working in 2D dimensions.
Computational cell is 6.18 x 21.8 x 0 with resolution 50
     block, center = (-1.585,0,0)
          size (3,1e+20,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (4,4,4)
     block, center = (4.16334e-17,-9.75,0)
          size (0.17,0.24,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
     block, center = (4.16334e-17,-9.45,0)
          size (0.17,0.24,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
     block, center = (4.16334e-17,-9.15,0)
          size (0.17,0.24,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
     block, center = (4.16334e-17,-8.85,0)
          siz

run 0 finished at t = 107.56 (10756 timesteps)
-----------
Initializing structure...
Halving computational cell along direction y
time for choose_chunkdivision = 0.000580072 s
Working in 2D dimensions.
Computational cell is 6.18 x 22 x 0 with resolution 50
     block, center = (-1.585,0,0)
          size (3,1e+20,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (4,4,4)
time for set_epsilon = 0.184662 s
-----------
Meep progress: 56.910000000000004/107.55555534362793 = 52.9% done in 4.0s, 3.6s to go
on time step 5691 (time=56.91), 0.000702942 s/step


run 0 finished at t = 107.56 (10756 timesteps)
-----------
Initializing structure...
Halving computational cell along direction y
time for choose_chunkdivision = 0.000757933 s
Working in 2D dimensions.
Computational cell is 6.18 x 22 x 0 with resolution 50
     block, center = (-1.585,0,0)
          size (3,1e+20,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (4,4,4)
     block, center = (4.16334e-17,-9.8,0)
          size (0.17,0.34,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
     block, center = (4.16334e-17,-9.4,0)
          size (0.17,0.34,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
     block, center = (4.16334e-17,-9,0)
          size (0.17,0.34,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
     block, center = (4.16334e-17,-8.6,0)
          size (0.17,

run 0 finished at t = 107.56 (10756 timesteps)
-----------
Initializing structure...
Halving computational cell along direction y
time for choose_chunkdivision = 0.000406981 s
Working in 2D dimensions.
Computational cell is 6.18 x 22 x 0 with resolution 50
     block, center = (-1.585,0,0)
          size (3,1e+20,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (4,4,4)
time for set_epsilon = 0.183024 s
-----------
Meep progress: 54.56/107.55555534362793 = 50.7% done in 4.0s, 3.9s to go
on time step 5456 (time=54.56), 0.000733273 s/step


run 0 finished at t = 107.56 (10756 timesteps)
-----------
Initializing structure...
Halving computational cell along direction y
time for choose_chunkdivision = 0.000457048 s
Working in 2D dimensions.
Computational cell is 6.18 x 22 x 0 with resolution 50
     block, center = (-1.585,0,0)
          size (3,1e+20,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (4,4,4)
     block, center = (4.16334e-17,-9.75,0)
          size (0.17,0.44,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
     block, center = (4.16334e-17,-9.25,0)
          size (0.17,0.44,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
     block, center = (4.16334e-17,-8.75,0)
          size (0.17,0.44,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
     block, center = (4.16334e-17,-8.25,0)
          size 

run 0 finished at t = 107.56 (10756 timesteps)
-----------
Initializing structure...
Padding y to even number of grid points.
Halving computational cell along direction y
time for choose_chunkdivision = 0.000571966 s
Working in 2D dimensions.
Computational cell is 6.18 x 22.3 x 0 with resolution 50
     block, center = (-1.585,0,0)
          size (3,1e+20,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (4,4,4)
time for set_epsilon = 0.183031 s
-----------
Meep progress: 58.08/107.55555534362793 = 54.0% done in 4.0s, 3.4s to go
on time step 5808 (time=58.08), 0.000688775 s/step


run 0 finished at t = 107.56 (10756 timesteps)
-----------
Initializing structure...
Padding y to even number of grid points.
Halving computational cell along direction y
time for choose_chunkdivision = 0.00062108 s
Working in 2D dimensions.
Computational cell is 6.18 x 22.3 x 0 with resolution 50
     block, center = (-1.585,0,0)
          size (3,1e+20,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (4,4,4)
     block, center = (4.16334e-17,-9.8,0)
          size (0.17,0.64,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
     block, center = (4.16334e-17,-9.1,0)
          size (0.17,0.64,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
     block, center = (4.16334e-17,-8.4,0)
          size (0.17,0.64,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
     block, center 

run 0 finished at t = 107.56 (10756 timesteps)
-----------
Initializing structure...
Padding y to even number of grid points.
Halving computational cell along direction y
time for choose_chunkdivision = 0.000581026 s
Working in 2D dimensions.
Computational cell is 6.18 x 22.7 x 0 with resolution 50
     block, center = (-1.585,0,0)
          size (3,1e+20,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (4,4,4)
time for set_epsilon = 0.189285 s
-----------
Meep progress: 52.49/107.55555534362793 = 48.8% done in 4.0s, 4.2s to go
on time step 5249 (time=52.49), 0.000762163 s/step


run 0 finished at t = 107.56 (10756 timesteps)
-----------
Initializing structure...
Padding y to even number of grid points.
Halving computational cell along direction y
time for choose_chunkdivision = 0.000634909 s
Working in 2D dimensions.
Computational cell is 6.18 x 22.7 x 0 with resolution 50
     block, center = (-1.585,0,0)
          size (3,1e+20,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (4,4,4)
     block, center = (4.16334e-17,-9.9,0)
          size (0.17,0.84,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
     block, center = (4.16334e-17,-9,0)
          size (0.17,0.84,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
     block, center = (4.16334e-17,-8.1,0)
          size (0.17,0.84,1e+20)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (1,1,1)
     block, center =